In [7]:
from pyspark.sql import SparkSession

# 1. Créer la session Spark
spark = SparkSession.builder \
    .appName("ConcatCSVtoParquet") \
    .config("spark.jars", "C:/Users/natha/Desktop/mysql-connector-j-9.0.0/mysql-connector-j-9.0.0/mysql-connector-j-9.0.0.jar") \
    .getOrCreate()

mysql_url = "jdbc:mysql://localhost:3306/projet_big_data"  # à adapter
mysql_properties = {
    "user": "root",
    "password": "",  # à adapter
    "driver": "com.mysql.cj.jdbc.Driver"
}

# Lecture des tables Silver depuis Bronze (format Parquet)
airlines_df = spark.read.parquet("silver/airlines")
airports_df = spark.read.parquet("silver/airports")
flights_df  = spark.read.parquet("silver/flights_table")


perf de vol

In [8]:
# from pyspark.sql.functions import col

# # Renommer les colonnes pour éviter les conflits
# airlines_renamed = airlines_df.withColumnRenamed("AIRLINE", "AIRLINE_NAME")
# airports_origin = airports_df.withColumnRenamed("IATA_CODE", "ORIGIN_CODE") \
#                              .withColumnRenamed("CITY", "ORIGIN_CITY") \
#                              .withColumnRenamed("STATE", "ORIGIN_STATE") \
#                              .withColumnRenamed("AIRPORT", "ORIGIN_AIRPORT_NAME")

# airports_dest = airports_df.withColumnRenamed("IATA_CODE", "DEST_CODE") \
#                            .withColumnRenamed("CITY", "DEST_CITY") \
#                            .withColumnRenamed("STATE", "DEST_STATE") \
#                            .withColumnRenamed("AIRPORT", "DEST_AIRPORT_NAME")

# # Construction du datamart
# dm_flight_performance = flights_df \
#     .join(airlines_renamed, flights_df.AIRLINE == airlines_renamed.IATA_CODE, "left") \
#     .join(airports_origin, flights_df.ORIGIN_AIRPORT == airports_origin.ORIGIN_CODE, "left") \
#     .join(airports_dest, flights_df.DESTINATION_AIRPORT == airports_dest.DEST_CODE, "left") \
#     .select(
#         "AIRLINE", "AIRLINE_NAME",
#         "ORIGIN_AIRPORT", "ORIGIN_CITY", "ORIGIN_STATE", "ORIGIN_AIRPORT_NAME",
#         "DESTINATION_AIRPORT", "DEST_CITY", "DEST_STATE", "DEST_AIRPORT_NAME",
#         "CANCELLED", "DIVERTED", "DISTANCE"
#     )

# # Écriture du datamart en Gold
# dm_flight_performance.write.mode("overwrite").parquet("gold/dm_flight_performance")
# dm_flight_performance.show(40)


airlines_renamed = airlines_df.withColumnRenamed("AIRLINE", "AIRLINE_NAME")
airports_origin = airports_df.withColumnRenamed("IATA_CODE", "ORIGIN_CODE") \
                             .withColumnRenamed("CITY", "ORIGIN_CITY") \
                             .withColumnRenamed("STATE", "ORIGIN_STATE") \
                             .withColumnRenamed("AIRPORT", "ORIGIN_AIRPORT_NAME")

airports_dest = airports_df.withColumnRenamed("IATA_CODE", "DEST_CODE") \
                           .withColumnRenamed("CITY", "DEST_CITY") \
                           .withColumnRenamed("STATE", "DEST_STATE") \
                           .withColumnRenamed("AIRPORT", "DEST_AIRPORT_NAME")

dm_flight_performance = flights_df \
    .join(airlines_renamed, flights_df.AIRLINE == airlines_renamed.IATA_CODE, "left") \
    .join(airports_origin, flights_df.ORIGIN_AIRPORT == airports_origin.ORIGIN_CODE, "left") \
    .join(airports_dest, flights_df.DESTINATION_AIRPORT == airports_dest.DEST_CODE, "left") \
    .select(
        "AIRLINE", "AIRLINE_NAME",
        "ORIGIN_AIRPORT", "ORIGIN_CITY", "ORIGIN_STATE", "ORIGIN_AIRPORT_NAME",
        "DESTINATION_AIRPORT", "DEST_CITY", "DEST_STATE", "DEST_AIRPORT_NAME",
        "CANCELLED", "DIVERTED", "DISTANCE"
    )
print(spark.sparkContext._jsc.sc().listJars())
try:
    spark._sc._gateway.jvm.java.lang.Class.forName("com.mysql.cj.jdbc.Driver")
    print("✅ Driver MySQL trouvé")
except Exception as e:
    print("❌ Driver MySQL non trouvé :", e)

dm_flight_performance.write.jdbc(
    url=mysql_url,
    table="dm_flight_performance",
    mode="overwrite",
    properties=mysql_properties
)
dm_flight_performance.show(40)


Vector()
❌ Driver MySQL non trouvé : An error occurred while calling z:java.lang.Class.forName.
: java.lang.ClassNotFoundException: com.mysql.cj.jdbc.Driver
	at java.base/jdk.internal.loader.BuiltinClassLoader.loadClass(BuiltinClassLoader.java:581)
	at java.base/jdk.internal.loader.ClassLoaders$AppClassLoader.loadClass(ClassLoaders.java:178)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:526)
	at java.base/java.lang.Class.forName0(Native Method)
	at java.base/java.lang.Class.forName(Class.java:315)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:566)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngin

Py4JJavaError: An error occurred while calling o304.jdbc.
: java.lang.ClassNotFoundException: com.mysql.cj.jdbc.Driver
	at java.base/java.net.URLClassLoader.findClass(URLClassLoader.java:476)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:593)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:526)
	at org.apache.spark.sql.execution.datasources.jdbc.DriverRegistry$.register(DriverRegistry.scala:46)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1(JDBCOptions.scala:103)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1$adapted(JDBCOptions.scala:103)
	at scala.Option.foreach(Option.scala:407)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:103)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:254)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scal

top routes

In [ ]:
# 

from pyspark.sql.functions import count

dm_top_routes = flights_df \
    .groupBy("ORIGIN_AIRPORT", "DESTINATION_AIRPORT") \
    .agg(count("*").alias("flight_count")) \
    .join(airports_origin, "ORIGIN_AIRPORT", "left") \
    .join(airports_dest, "DESTINATION_AIRPORT", "left") \
    .select(
        "ORIGIN_AIRPORT", "ORIGIN_CITY", "ORIGIN_AIRPORT_NAME",
        "DESTINATION_AIRPORT", "DEST_CITY", "DEST_AIRPORT_NAME",
        "flight_count"
    )

dm_top_routes.write.jdbc(
    url=mysql_url,
    table="dm_top_routes",
    mode="overwrite",
    properties=mysql_properties
)


+-------+--------------------+--------------+--------------------+------------+--------------------+-------------------+-----------------+----------+--------------------+---------+--------+--------+
|AIRLINE|        AIRLINE_NAME|ORIGIN_AIRPORT|         ORIGIN_CITY|ORIGIN_STATE| ORIGIN_AIRPORT_NAME|DESTINATION_AIRPORT|        DEST_CITY|DEST_STATE|   DEST_AIRPORT_NAME|CANCELLED|DIVERTED|DISTANCE|
+-------+--------------------+--------------+--------------------+------------+--------------------+-------------------+-----------------+----------+--------------------+---------+--------+--------+
|     UA|United Air Lines ...|           ORD|             Chicago|          IL|Chicago O'Hare In...|                LGA|         New York|        NY|LaGuardia Airport...|        0|       0|     733|
|     UA|United Air Lines ...|           EWR|              Newark|          NJ|Newark Liberty In...|                SFO|    San Francisco|        CA|San Francisco Int...|        0|       0|    2565|
|    

air port summary

In [ ]:
# from pyspark.sql.functions import sum as _sum

# # Statistiques agrégées par aéroport d'origine
# airport_stats = flights_df.groupBy("ORIGIN_AIRPORT") \
#     .agg(
#         count("*").alias("total_flights"),
#         _sum("CANCELLED").alias("cancelled_flights"),
#         _sum("DIVERTED").alias("diverted_flights")
#     )

# # Ajout des infos de l'aéroport
# dm_airport_summary = airport_stats \
#     .join(airports_df, airport_stats.ORIGIN_AIRPORT == airports_df.IATA_CODE, "left") \
#     .select(
#         "ORIGIN_AIRPORT",
#         "AIRPORT", "CITY", "STATE",
#         "total_flights", "cancelled_flights", "diverted_flights"
#     )

# # Sauvegarde
# dm_airport_summary.write.mode("overwrite").parquet("gold/dm_airport_summary")

# dm_airport_summary.show(40)


from pyspark.sql.functions import sum as _sum

airport_stats = flights_df.groupBy("ORIGIN_AIRPORT") \
    .agg(
        count("*").alias("total_flights"),
        _sum("CANCELLED").alias("cancelled_flights"),
        _sum("DIVERTED").alias("diverted_flights")
    )

dm_airport_summary = airport_stats \
    .join(airports_df, airport_stats.ORIGIN_AIRPORT == airports_df.IATA_CODE, "left") \
    .select(
        "ORIGIN_AIRPORT",
        "AIRPORT", "CITY", "STATE",
        "total_flights", "cancelled_flights", "diverted_flights"
    )

dm_airport_summary.write.jdbc(
    url=mysql_url,
    table="dm_airport_summary",
    mode="overwrite",
    properties=mysql_properties
)



+--------------+--------------------+-------------------+-----+-------------+-----------------+----------------+
|ORIGIN_AIRPORT|             AIRPORT|               CITY|STATE|total_flights|cancelled_flights|diverted_flights|
+--------------+--------------------+-------------------+-----+-------------+-----------------+----------------+
|           BGM|Greater Binghamto...|         Binghamton|   NY|          262|              3.0|             0.0|
|           INL|Falls Internation...|International Falls|   MN|          574|              9.0|             9.0|
|           PSE|   Mercedita Airport|              Ponce|   PR|          749|             11.0|             0.0|
|         12888|                NULL|               NULL| NULL|           53|              1.0|             0.0|
|           MSY|Louis Armstrong N...|        New Orleans|   LA|        38804|            411.0|           101.0|
|           PPG|Pago Pago Interna...|          Pago Pago|   AS|          107|              0.0| 